In [ ]:
import json
import requests

In [ ]:
# path to query terms from Cultural AI GitHub
# EN
url_en = "https://github.com/cultural-ai/wordsmatter/raw/main/query_terms_cont_en.json"
# NL
url_nl = "https://github.com/cultural-ai/wordsmatter/raw/main/query_terms_cont_nl.json"

In [ ]:
getting_en = requests.get(url_en)
wordforms_en = json.loads(getting_en.text)

In [ ]:
getting_nl = requests.get(url_nl)
wordforms_nl = json.loads(getting_nl.text)

In [ ]:
# merging query terms together in one dict
# {'en':{'lemma':[all wordforms inclusing lemma]}}

query_terms = {}
query_terms['en'] = wordforms_en
query_terms['nl'] = wordforms_nl

In [ ]:
for values in query_terms.values():
    for lemma, wordforms in values.items():
        wordforms.append(lemma)

In [ ]:
# saving all query terms
with open('query_terms.json', 'w') as jf:
    json.dump(query_terms, jf)

In [ ]:
# stats
en_total = 0
en_lemmas = 0
nl_total = 0
nl_lemmas = 0

for lemma, wordforms in query_terms['en'].items():
    en_lemmas += 1
    en_total += len(wordforms)
    
for lemma, wordforms in query_terms['nl'].items():
    nl_lemmas += 1
    nl_total += len(wordforms)
    
print("EN:",en_lemmas,en_total)
print("NL:",nl_lemmas,nl_total)